In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv('F:/Books/Machine Learning/projects/ASHREE(Kaggle)/train.csv')

In [3]:
train_data.shape

(20216100, 4)

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20216100 entries, 0 to 20216099
Data columns (total 4 columns):
building_id      int64
meter            int64
timestamp        object
meter_reading    float64
dtypes: float64(1), int64(2), object(1)
memory usage: 616.9+ MB


In [ ]:
#nthread = 4,
#          learning_rate = 0.05,
#          num_leaves = 40,
#          colsample_bytree = 0.85,
#          lambda = 2)

In [6]:
train_data.head()

,building_id,meter,timestamp,meter_reading
0,0,0,2016-01-01 00:00:00,0.0
1,1,0,2016-01-01 00:00:00,0.0
2,2,0,2016-01-01 00:00:00,0.0
3,3,0,2016-01-01 00:00:00,0.0
4,4,0,2016-01-01 00:00:00,0.0


In [15]:
# shape( 20,216,100,   4)
# total 1449 building_id
# 8784 timestamp unique
# 1688175 meter_reading unique   Target_Variable
# meter 4
train_data.meter.nunique()

4

In [17]:
meta_data = pd.read_csv('F:/Books/Machine Learning/projects/ASHREE(Kaggle)/building_metadata.csv')

In [19]:
meta_data.head()

,site_id,building_id,primary_use,square_feet,year_built,floor_count
0,0,0,Education,7432,2008.0,NaN
1,0,1,Education,2720,2004.0,NaN
2,0,2,Education,5376,1991.0,NaN
3,0,3,Education,23685,2002.0,NaN
4,0,4,Education,116607,1975.0,NaN


In [47]:
# shape (1449,6)
# floor_count 18 [1.2.3.4..5]
# year_build 116 [years] 1900 - 2017
# square_feet 1397 
# primaryUse  16 [string names]
# building_id 1449 
# site_id     16 [0,1,2,3,]
meta_data.site_id.value_counts().head(),


(3     274
 13    154
 2     135
 15    124
 9     124
 Name: site_id, dtype: int64,)

In [48]:
weather_data = pd.read_csv('F:/Books/Machine Learning/projects/ASHREE(Kaggle)/weather_train.csv')

In [49]:
weather_data.shape

(139773, 9)

In [50]:
weather_data.head()

,site_id,timestamp,air_temperature,cloud_coverage,dew_temperature,precip_depth_1_hr,sea_level_pressure,wind_direction,wind_speed
0,0,2016-01-01 00:00:00,25.0,6.0,20.0,NaN,1019.7,0.0,0.0
1,0,2016-01-01 01:00:00,24.4,NaN,21.1,-1.0,1020.2,70.0,1.5
2,0,2016-01-01 02:00:00,22.8,2.0,21.1,0.0,1020.2,0.0,0.0
3,0,2016-01-01 03:00:00,21.1,2.0,20.6,0.0,1020.1,0.0,0.0
4,0,2016-01-01 04:00:00,20.0,2.0,20.0,-1.0,1020.0,250.0,2.6


In [70]:
# shape (139,773  9)

# site_id      16
# timestamp    8784
# air_temp     619
# cloud_cover  10
# dew_temp     522
# precip_depth 128
# sea_level    709
# winddirecti  43
# windspeed    58


In [71]:
test_data = pd.read_csv('F:/Books/Machine Learning/projects/ASHREE(Kaggle)/test.csv')

In [74]:
test_data.shape

(41697600, 4)

In [75]:
test_data.head()

,row_id,building_id,meter,timestamp
0,0,0,0,2017-01-01 00:00:00
1,1,1,0,2017-01-01 00:00:00
2,2,2,0,2017-01-01 00:00:00
3,3,3,0,2017-01-01 00:00:00
4,4,4,0,2017-01-01 00:00:00


In [79]:
# shape (41,697,600,  4)
# row_id  [ all unique ]
# building_id
# meter
# timestamp

41697600

### Testing data insights
We have only 3 feature `building_id`, `meter`, `timestamp` to predict target variable.

After prediction we save those values in sample_submission as using `row_id`

In [76]:
sample_submission = pd.read_csv('F:/Books/Machine Learning/projects/ASHREE(Kaggle)/sample_submission.csv')

In [77]:
# Sample submission
# (41,697,600  ,  2)
# row_id,    
# meter_reading

(41697600, 2)

In [78]:
sample_submission.head()

,row_id,meter_reading
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [81]:
## Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.
                      format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [82]:
train_df = reduce_mem_usage(train_data)

Mem. usage decreased to 289.19 Mb (53.1% reduction)


In [87]:
weather_data_df = reduce_mem_usage(weather_data)

Mem. usage decreased to  3.07 Mb (68.1% reduction)


In [95]:
sample_submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41697600 entries, 0 to 41697599
Data columns (total 2 columns):
row_id           int32
meter_reading    int8
dtypes: int32(1), int8(1)
memory usage: 198.8 MB
